<a href="https://colab.research.google.com/github/younesabdolmalaky/A-Dual-Channel-Approach-for-Farsi-Text-Classification-using-Transfer-Learning-Techniques/blob/main/notebooks/CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [32]:
!unzip /content/drive/MyDrive/persian-sentiment-analysis/train_pad_sequences.zip

Archive:  /content/drive/MyDrive/persian-sentiment-analysis/train_pad_sequences.zip
  inflating: content/train_pad_sequences.pickle  


In [2]:
from tensorflow.keras.preprocessing.text import Tokenizer, text_to_word_sequence
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import callbacks
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input , Dense
from tensorflow.keras.utils import plot_model
from tensorflow.keras.utils import Sequence
import bz2
import numpy as np
import pickle
import mmap 
import re

In [3]:
MAX_FEATURES = 12000
sequences = Input(shape=(255,))
embedded = layers.Embedding(MAX_FEATURES, 32)(sequences)
x = layers.Conv1D(64, 3, activation='relu')(embedded)
x = layers.BatchNormalization()(x)
x = layers.MaxPool1D(3)(x)
x = layers.Conv1D(32, 5, activation='relu')(x)
x = layers.BatchNormalization()(x)
x = layers.MaxPool1D(5)(x)
x = layers.Conv1D(16, 5, activation='relu')(x)
x = layers.GlobalMaxPool1D()(x)
x = layers.Flatten()(x)
x = layers.Dense(64, activation='relu')(x)
outputs = Dense(1, activation='sigmoid')(x)

In [4]:
model = Model(inputs=sequences, outputs=outputs)

In [5]:
mch = callbacks.ModelCheckpoint('../models/fianl-model-cnn.h5' , monitor='accuracy' , mode ='max' , save_best_only=True)

In [6]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [7]:
class DataGenerator(Sequence):
    def __init__(self , dataset_size ,x_file, y_file, batch_size=32):
        self.dataset_size = dataset_size
        self.x_file = x_file
        self.y_file = y_file
        self.batch_size = batch_size


        with open(x_file, 'rb') as f:
          mm = mmap.mmap(f.fileno(), 0, access=mmap.ACCESS_READ)
          self.arr_train = np.load(mm , allow_pickle=True)
        

        with open(y_file, 'rb') as f:
          mm = mmap.mmap(f.fileno(), 0, access=mmap.ACCESS_READ)
          self.arr_y = np.load(mm , allow_pickle=True)


    def __len__(self):
        return int(np.ceil((self.dataset_size) / float(self.batch_size)))

    def __getitem__(self, idx):
        batch_X = self.arr_train[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_y = self.arr_y[idx * self.batch_size:(idx + 1) * self.batch_size]

        return batch_X, batch_y
batch_size = 8192
dataset_size_train = 3600000
dataset_size_test = 400000
train_generator = DataGenerator(dataset_size_train,"/content/drive/MyDrive/persian-sentiment-analysis/train_pad_sequences.pickle", "/content/drive/MyDrive/persian-sentiment-analysis/train_labels.pickle", batch_size=batch_size)
test_generator = DataGenerator(dataset_size_test,"/content/drive/MyDrive/persian-sentiment-analysis/test_pad_sequences.pickle", "/content/drive/MyDrive/persian-sentiment-analysis/test_labels.pickle", batch_size=batch_size)

In [8]:
model.fit(train_generator, steps_per_epoch=len(train_generator), epochs=100 , validation_data=test_generator, validation_steps=len(test_generator),callbacks=[mch])

Epoch 1/100
440/440 [==============================] - 138s 285ms/step - loss: 0.2407 - accuracy: 0.8959 - val_loss: 0.3597 - val_accuracy: 0.9171
Epoch 2/100
440/440 [==============================] - 110s 251ms/step - loss: 0.1554 - accuracy: 0.9415 - val_loss: 0.1577 - val_accuracy: 0.9407
Epoch 3/100
440/440 [==============================] - 109s 248ms/step - loss: 0.1399 - accuracy: 0.9483 - val_loss: 0.1503 - val_accuracy: 0.9441
Epoch 4/100
440/440 [==============================] - 110s 250ms/step - loss: 0.1299 - accuracy: 0.9526 - val_loss: 0.1597 - val_accuracy: 0.9399
Epoch 5/100
440/440 [==============================] - 112s 255ms/step - loss: 0.1225 - accuracy: 0.9558 - val_loss: 0.1512 - val_accuracy: 0.9448
Epoch 6/100
440/440 [==============================] - 110s 250ms/step - loss: 0.1164 - accuracy: 0.9585 - val_loss: 0.1845 - val_accuracy: 0.9316
Epoch 7/100
440/440 [==============================] - 110s 250ms/step - loss: 0.1107 - accuracy: 0.9609 - val_loss: 0